## 케라스 모델을 만드는 여러 방법
- `Sequential API` + 내장층
  - 초보자, 간단한 모델
  - 하나의 파이썬 리스트
  - 단순히 층을 쌓기만 가능
- `Functional API`
  - "+ 내장층" : 일반적인 문제를 다루는 엔지니어
  - "+ 사용자 정의 층, 사용자 정의 지표, 사용자 정의 손실, etc." : 보기 드문 문제를 위해 맞춤형 솔루션이 필요한 엔지니어
  - 그래프 같은 모델 구조
  - 사용성, 유연성 사이 적절한 중간
  - 가장 널리 사용되는 모델 구축 API
- `Model subclassing`
  - 연구자
  - 모든 것을 밑바닥부터 직접 만드는 저수준 방법
  - 모든 상세한 내용을 완전히 제어하고 싶은 경우
  - 케라스 내장 기능을 사용하지 않기 때문에 실수 가능성 높음

### `Sequential` 모델

In [5]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")
])

model

In [ ]:
model.weights

입력 크기를 알기 전까지 가중치를 만들 수 가 없음  
- 해당 Sequential 모델은 아직 어떤 가중치도 가지고 있지 않음
- 가중치를 생성하기 위해서는 어떤 데이터로 호출하거나 입력 크기를 지정하여 `build()` 호출해야 함

In [ ]:
model.build(input_shape=(None, 3))
model.weights

In [ ]:
model.summary()

In [ ]:
# 하나씩 추가해보기
model = keras.Sequential(name="my_example_model")
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))
model.build((None, 3))
model.summary()

In [ ]:
model = keras.Sequential()
model.summary() # build() 를 호출하지 않아서 정보가 아무것도 없음

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(3,))) # 하지만 Input 클래스를 사용해주면 summary 가능
model.add(layers.Dense(64, activation="relu"))
model.summary()

**Sequential 모델**  
- 쉬움
- 적용 가능한 곳이 제한적
- 하나의 입력과 하나의 출력
- 순서대로 층을 쌓은 모델만 표현 가능
- 다중 입력, 다중 출력, 비선형적 구조에 사용 불가

### 함수형 API
- 다중 입력, 다중 출력, 비선형적 구조에 사용

#### 간단한 예제

In [ ]:
# Input 클래스 객체 정의
inputs = keras.Input(shape=(3,), name="my_input")
inputs.shape, inputs.dtype # 모델이 처리할 데이터 크기와 dtype 정보

**Symbolic tensor**  
- 실제 데이터를 가지고 있지 않지만 사용할 때 모델이 보게 될 데이터 텐서의 사양이 인코딩 된 텐서
- 미래의 데이터 텐서를 나타냄

In [ ]:
# 층을 만들고 입력 호출
features = layers.Dense(64, activation="relu")(inputs)
features.shape # symbolic tensor로 호출 : 크기와 dtype 정보가 업데이트된 새로운 심볼릭 텐서를 반환

In [ ]:
# 최종 출력
outputs = layers.Dense(10, activation="softmax")(features)
# 모델 객체 생성 : 입력과 출력을 Model 클래스에 전달
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

#### 다중 입력, 다중 출력 모델
**고객 이슈 티켓 할당 시스템**  
- 입력
  - 이슈 티켓의 제목(텍스트)
  - 이슈 티켓의 본문(텍스트)
  - 사용자가 추가한 태그(범주형 입력 : 원-핫 인코딩 가정)
- 출력
  - 이슈 티켓의 우선순위 점수, 0과 1 사이의 스칼라(시그모이드 출력)
  - 이슈 티켓을 처리해야 할 부서(전체 부서 집합에 대한 소프트맥스 출력)

In [ ]:
vocabulary_size = 10000
num_tags = 100
num_departments = 4

# Define model inputs
title = keras.Input(shape=(vocabulary_size,), name="title")
text_body = keras.Input(shape=(vocabulary_size,), name="text_body")
tags = keras.Input(shape=(num_tags,), name="tags")

# 입력 특성을 하나의 텐서 features로 연결
features = layers.Concatenate()([title, text_body, tags])
# 중간층을 적용하여 입력 특성을 더 풍부한 표현으로 재결합
features = layers.Dense(64, activation="relu")(features)

# Define model outputs
priority = layers.Dense(1, activation="sigmoid", name="priority")(features)
department = layers.Dense(num_departments, activation="softmax", name="department")(features)

# Create model
model = keras.Model(inputs=[title, text_body, tags],
                    outputs=[priority, department])

In [ ]:
model.summary()

#### 다중 입력, 다중 출력 모델 훈련하기

In [ ]:
import numpy as np

num_samples = 1280

# input dummy
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

# target dummy
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit([title_data, text_body_data, tags_data],
          [priority_data, department_data],
          epochs=1)

model.evaluate([title_data, text_body_data, tags_data],
               [priority_data, department_data])

priority_preds, department_preds = model.predict([title_data, text_body_data, tags_data])

#### 부여한 이름을 활용해서 훈련하기
- Input 객체와 출력 층에 데이터를 딕셔너리로 전달

In [ ]:
model.compile(optimizer="rmsprop",
              loss={"priority": "mean_squared_error", "department": "categorical_crossentropy"},
              metrics={"priority": ["mean_absolute_error"], "department": ["accuracy"]})

model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          {"priority": priority_data, "department": department_data},
          epochs=1)

model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
               {"priority": priority_data, "department": department_data})

priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

#### 함수형 API의 장점: 층 연결 구조 활용하기
- 모델 시각화
- 특성 추출

In [ ]:
# 모델의 topology를 시각화하기
keras.utils.plot_model(model, "ticket_classifier.png")

In [ ]:
# 각 층의 입력 크기 추가
keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)

In [ ]:
model.layers

In [ ]:
model.layers[3].input

In [ ]:
model.layers[3].output

#### 중간 층의 출력을 재사용해 새로운 모델 만들기
- feature extraction 수행 후 다른 모델에서 중간 특성을 재사용
- 이전 모델에 또 다른 출력을 추가
  - 이슈 티켓이 해결되는 데 걸리는 시간(난이도) 추정

In [ ]:
# layers[4]는 중간 Dense layer
features = model.layers[4].output
difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty]
)

In [ ]:
keras.utils.plot_model(new_model, "updated_ticket_classifier.png", show_shapes=True)

### Model subclassing
- `__init__()` 메소드에서 모델이 사용할 층을 정의
- `call()` 메소드에서 앞서 만든 층을 사용하여 모델의 정방향 패스를 정의
- 서브클래스의 객체를 만들고 데이터와 함께 호출하여 가중치 만들기

In [ ]:
class CustomerTicketModel(keras.Model):

    def __init__(self, num_departments):
        super().__init__()
        self.concat_layer = layers.Concatenate()
        
        # Define layers in constructer
        self.mixing_layer = layers.Dense(64, activation="relu")
        self.priority_scorer = layers.Dense(1, activation="sigmoid")
        self.department_classifier = layers.Dense(
            num_departments, activation="softmax")

    def call(self, inputs):
        # Define forward pass
        title = inputs["title"]
        text_body = inputs["text_body"]
        tags = inputs["tags"]

        features = self.concat_layer([title, text_body, tags])
        features = self.mixing_layer(features)
        priority = self.priority_scorer(features)
        department = self.department_classifier(features)

        # return outputs
        return priority, department

In [ ]:
model = CustomerTicketModel(num_departments=4)
model.weights

In [ ]:
# 어떤 데이터로 처음 호출할 때 가중치 생성
priority, department = model(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})
model.weights

In [ ]:
model.compile(optimizer="rmsprop",
              loss=["mean_squared_error", "categorical_crossentropy"],
              metrics=[["mean_absolute_error"], ["accuracy"]])

model.fit({"title": title_data,
           "text_body": text_body_data,
           "tags": tags_data},
          [priority_data, department_data],
          epochs=1)

model.evaluate({"title": title_data,
                "text_body": text_body_data,
                "tags": tags_data},
               [priority_data, department_data])

priority_preds, department_preds = model.predict({"title": title_data,
                                                  "text_body": text_body_data,
                                                  "tags": tags_data})

**서브클래싱 모델에서는 지원하지 않는 것들이 많음**
- 서브클래싱 모델은 한 덩어리의 bytecode
- 층이 연결되는 방식이 `call() 메소드 안에 숨겨짐
  - `summary()` 메서드 사용 불가
  - `plot_model()` 사용 불가

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "subclassing_ticket_classifier.png", show_shapes=True)

#### 여러 방식을 혼합하여 사용하기
- 케라스 API로 만든 모델은 서로 혼합 가능
- e.g. 함수형 모델에서 서브클래싱 층이나 모델을 사용 가능

**서브클래싱한 모델을 포함하는 함수형 모델**

In [ ]:
# subclassing classifier
class Classifier(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        if num_classes == 2:
            num_units = 1
            activation = "sigmoid"
        else:
            num_units = num_classes
            activation = "softmax"
        self.dense = layers.Dense(num_units, activation=activation)

    def call(self, inputs):
        return self.dense(inputs)

inputs = keras.Input(shape=(3,))
features = layers.Dense(64, activation="relu")(inputs)
outputs = Classifier(num_classes=10)(features) # subclassing classifier
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "mixing_subclassing_classifier.png", show_shapes=True)

**함수형 모델을 포함하는 서브클래싱 모델**

In [ ]:
inputs = keras.Input(shape=(64,))
outputs = layers.Dense(1, activation="sigmoid")(inputs)
binary_classifier = keras.Model(inputs=inputs, outputs=outputs)

class MyModel(keras.Model):

    def __init__(self, num_classes=2):
        super().__init__()
        self.dense = layers.Dense(64, activation="relu")
        # 함수형 모델 classififier
        self.classifier = binary_classifier

    def call(self, inputs):
        # 함수형 모델 inputs & features
        features = self.dense(inputs)
        return self.classifier(features)

model = MyModel()

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "mixing_functional_model.png", show_shapes=True)

- DAG(Directed Asyclic Graph)로 표현 가능 : 함수형 API 사용이 유용
- DAG로 표현이 불가능 : 서브클래싱 모델 사용
- 서브클래싱 층을 포함한 함수형 모델
  - 함수형 API의 장점 유지
  - 높은 개발 유연성 제공

## 내장된 훈련 루프와 평가 루프 사용하기
- 복잡성의 단계적 공개 원칙
  - 아주 쉬운 것부터 매우 유연한 것까지 한 번에 한 단계씩 전체 워크플로우에 접근 가능
  - 모델 훈련에도 적용됨
  - 간단하게는 `fit()` 메소드를 호출 ~ 고급으로는 밑바닥부터 새로운 훈련 알고리즘을 작성 가능

### 표준 워크플로우
- `compile()`, `fit()`, `evaluate()`, `predict()`

In [1]:
# from tensorflow.keras.datasets import mnist
import numpy as np

# 재사용이 가능한 모델 정의 함수
def get_mnist_model():
    inputs = keras.Input(shape=(28*28,))
    features = layers.Dense(512, activation="relu")(inputs)
    features = layers.Dropout(0.5)(features)
    outputs = layers.Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

# (images, labels), (test_images, test_labels) = mnist.load_data()

data = np.load('/kaggle/input/keras-mnist/mnist.npz')
images = data['x_train']
labels = data['y_train']
test_images = data['x_test']
test_labels = data['y_test']

images.shape, labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [2]:
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255

images.shape, test_images.shape

((60000, 784), (10000, 784))

In [3]:
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

train_images.shape, val_images.shape, train_labels.shape, val_labels.shape

((50000, 784), (10000, 784), (50000,), (10000,))

In [6]:
model = get_mnist_model()
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.fit(train_images, train_labels, epochs=3, validation_data=(val_images, val_labels))

test_metrics = model.evaluate(test_images, test_labels)
predictions = model.predict(test_images)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8665 - loss: 0.4489 - val_accuracy: 0.9588 - val_loss: 0.1443
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9504 - loss: 0.1701 - val_accuracy: 0.9681 - val_loss: 0.1144
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9625 - loss: 0.1315 - val_accuracy: 0.9726 - val_loss: 0.0993
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9702 - loss: 0.1131
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


### 사용자 정의 지표 만들기
- metric(지표) : 모델의 성능을 측정하는 열쇠
- keras.metrics.Metric 클래스를 상속하여 간단히 커스텀마이징 가능
- 층과 마찬가지로 metric은 텐서플로우 변수에 내부 상태를 저장
- 층과 다른 부분은 이 변수가 역전파로 업데이트되지 않는다는 점
  - `update_state()` 메소드 안에 상태 업데이트 로직을 작성해야 함

In [7]:
import tensorflow as tf

class RootMeanSquaredError(keras.metrics.Metric): # metric 클래스 상속
    def __init__(self, name="rmse", **kwargs):
        super().__init__(name=name, **kwargs)
        
        ### Define state variable ###
        # 상태 변수 mse_sum는 모든 배치에서 계산된 MSE의 합계를 저장, 0.0 스칼라
        self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
        # 상태 변수 total_samples 현재까지 처리된 전체 샘플의 개수, 0 정수형 스칼라
        self.total_samples = self.add_weight(name="total_samples", initializer="zeros", dtype="int32")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # y_pred : 각 클래스에 대한 확률값
        # y_true : 정수형 레이블 -> y_pred shape에 맞춰서 one hot encoding
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
        # 현재 배치에서 발생한 y_true(target)와 y_pred(predition)의 mse를 계산
        mse = tf.reduce_sum(tf.square(y_true - y_pred))

        ### update state variables ###
        # 상태 변수 mse_sum에 각 배치에서 발생한 mse를 누적
        self.mse_sum.assign_add(mse)
        # 상태 변수 total_samples에 현재 배치의 샘플 개수를 구하여 누적
        num_samples = tf.shape(y_pred)[0]
        self.total_samples.assign_add(num_samples)

    def result(self):
        ### return the current metric value ###
        # self.mse_sum / tf.cast(self.total_samples, tf.float32) -> average mse
        # tf.sqrt() -> rmse
        return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

    def reset_state(self):
        # 객체를 다시 생성하지 않고 상태를 초기화
        #  - metric object 하나를 서로 다른 훈련 반복에 사용
        #  - 혹은 훈련과 평가에 모두 사용
        self.mse_sum.assign(0.)
        self.total_samples.assign(0)

In [8]:
model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy", RootMeanSquaredError()])

model.fit(train_images, train_labels, epochs=3,
          validation_data=(val_images, val_labels))
test_metrics = model.evaluate(test_images, test_labels)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8626 - loss: 0.4553 - rmse: 0.4418 - val_accuracy: 0.9583 - val_loss: 0.1465 - val_rmse: 0.2538
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9510 - loss: 0.1675 - rmse: 0.2729 - val_accuracy: 0.9634 - val_loss: 0.1314 - val_rmse: 0.2373
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9626 - loss: 0.1365 - rmse: 0.2406 - val_accuracy: 0.9708 - val_loss: 0.1092 - val_rmse: 0.2142
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9679 - loss: 0.1148 - rmse: 0.2214


```
Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8626 - loss: 0.4553 - **rmse**: 0.4418 - val_accuracy: 0.9583 - val_loss: 0.1465 - val_rmse: 0.2538
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9510 - loss: 0.1675 - **rmse**: 0.2729 - val_accuracy: 0.9634 - val_loss: 0.1314 - val_rmse: 0.2373
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9626 - loss: 0.1365 - **rmse**: 0.2406 - val_accuracy: 0.9708 - val_loss: 0.1092 - val_rmse: 0.2142
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9679 - loss: 0.1148 - **rmse**: 0.2214
```

진행 표시줄에서 rmse도 함께 확인 가능

### 콜백 사용하기
- 대규모 데이터셋에서 `model.fit()` 메소드를 사용하여 수십 번의 에포크를 실행하는 것은 비효율적
- 케라스 callback을 이용하여 `model.fit()` 호출을 스스로 판단하고 동적으로 결정하게 수정 가능
- **Callback** : `fit()` 메소드에서 호출 시 모델에 전달되는 객체
  - 아래 6가지 메소드를 구현한 클래스 객체
    - `on_epoch_begin(epoch, logs)`, `on_epoch_end(epoch, logs)`
    - `on_batch_begin(epoch, logs)`, `on_batch_end(epoch, logs)`
    - `on_train_begin(epoch, logs)`, `on_train_end(epoch, logs)`
  - 모델의 상태와 성능에 대한 모든 정보에 접근 가능
  - 훈련 중지, 모델 저장, 가중치 적재 또는 모델 상태 변경 등 처리 가능
- Callback 예시
  - 모델 checkpoint 저장
    - 훈련 동안 어떤 지점의 모델 가중치를 저장
  - early stopping
    - validation loss가 더 이상 향상되지 않을 때 훈련 중지 후 가장 좋은 모델 저장
  - 훈련 동안 hyperparamter 값을 동적으로 조정
    - e.g. 옵티마이저의 학습률
  - 훈련과 검증 지표를 로그에 기록, 혹은 모델이 학습한 표현이 업데이트 되면 시각화
    - e.g. `fit()`의 진행 표시줄

#### ModelCheckpoint와 EarlyStopping callback
- 정해진 에포크 내에 모니터링 지표가 향상되지 않을 때 훈련 중지
- ModelCheckpoint callback과 함께 사용
- 선택적으로 지금까지 가장 좋은 모델만 저장
- 그럼 에포크 끝에서 최고의 성능을 낸 모델을 찾기 쉽겠지..

In [12]:
# callback list : 몇 개의 콜백이라도 전달 가능
callbacks_list = [
    keras.callbacks.EarlyStopping( # 성능 향상이 멈추면 훈련 중지
        monitor="val_accuracy",    # 모델 검증 정확도를 모니터링
        patience=2,                # 두 번의 에포크 동안 정확도가 향상되지 않으면 훈련 중지
    ),
    keras.callbacks.ModelCheckpoint(   # 매 에포크 끝에서 현재 가중치 저장
        filepath="/kaggle/working/checkpoint_path.keras", # 모델 파일의 저장 경로
        monitor="val_loss",            # 검증 손실이 좋아지지 않으면
        save_best_only=True,           # 모델 파일을 덮어쓰지 않는다 -> 가장 좋은 모델이 저장됨
    )
]

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"]) # val_accuracy를 모니터링하기 때문에 검증 지표에 accuracy가 있어야 함

model.fit(train_images, train_labels,
          epochs=10,
          callbacks=callbacks_list, # callbacks_list 전달
          validation_data=(val_images, val_labels)) # 콜백이 val_accuracy & val_loss를 모니터링하므로
                                                    # validation_data가 꼭 필요

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8634 - loss: 0.4545 - val_accuracy: 0.9565 - val_loss: 0.1486
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9496 - loss: 0.1692 - val_accuracy: 0.9657 - val_loss: 0.1212
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9627 - loss: 0.1316 - val_accuracy: 0.9721 - val_loss: 0.0966
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9681 - loss: 0.1146 - val_accuracy: 0.9737 - val_loss: 0.1061
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9710 - loss: 0.0994 - val_accuracy: 0.9761 - val_loss: 0.0967
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9740 - loss: 0.0944 - val_accuracy: 0.9767 - val_loss: 0.0953
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9769 - loss: 0.0874 - val_accuracy: 0.9778 - val_loss: 0.0943
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9781 - loss: 0.0836 - 

**Checkpoint 파일 저장 에러**  
```
ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=/kaggle/working/checkpoint_path.h5
```
- `.h5`를 `.keras`로 변경
- weight만 저장하고 싶을 경우 `.weights.h5` 사용